* importing the module 
* setting the seed 
* reading the train input

In [85]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import numpy as np
np.random.seed(0)
from nltk.stem import WordNetLemmatizer
np.random.seed(0)
sent_train=pd.read_csv('train.csv',encoding="latin-1")
sent_test=pd.read_csv('test.csv',encoding="latin-1")

* intialising the tokenizer


In [86]:
tokenizer = RegexpTokenizer(r'\w+')

* tokenizing and transforming the sentence by 
    * omitting common stopwords
    * removing punctuations
    * removing special symbols

In [87]:
import re
stopwords = nltk.corpus.stopwords.words('english')
stopwords_lower = [s.lower() for s in stopwords]
def text_preprocessing(str_input): 
     #tokenization, remove punctuation, lemmatization
     words=tokenizer.tokenize(str_input)
     # remove symbols, websites, email addresses 
     words = [re.sub(r'[^A-Za-z@]', ' ', word) for word in words] 
     words = [re.sub(r'\S+com', ' ', word) for word in words]
     words = [re.sub(r'\S+@\S+', ' ', word) for word in words] 
     words = [word for word in words if word!=' ']
     words = [word for word in words if len(word)!=0] 
     #remove stopwords     
     words=[word.lower() for word in words if word.lower() not in stopwords_lower and word.strip() != ""]
     #combine a list into one string   
     string = ' '.join(words)
     return string

sent_train['modified_sent']= sent_train['Sentence'].apply(text_preprocessing)
sent_test['modified_sent']= sent_test['Sentence'].apply(text_preprocessing)

lemmatizer = WordNetLemmatizer()
def lemmatize_words(str):
    words = [lemmatizer.lemmatize(word) for word in str.split()]
    return ' '.join(words)
    
sent_train['modified_sent'] = sent_train.modified_sent.apply(lemmatize_words)
sent_test['modified_sent'] = sent_test.modified_sent.apply(lemmatize_words)

* Making bog type of matrix using TfidfVectorizer
* loading the training dataset

In [88]:
import sklearn
from  sklearn.feature_extraction.text import TfidfVectorizer

text = [text for text in sent_train.modified_sent]
vector_text = TfidfVectorizer(stop_words='english')
vector_text.fit(text)
vector = vector_text.transform(text)
x_train=(vector.toarray())
y_train = [i for i in sent_train.Sentiment]

* choosing the model for classification
* splitting the dataset into train and cross-validation

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
train_x, test_x, train_y, test_y = train_test_split(x_train, y_train, test_size=0.3)

* fitting the model on training dataset
* checking the f1-score on cross validation dataset

In [90]:
clf = MultinomialNB().fit(train_x, train_y)
predict_y=clf.predict(test_x)
print(f1_score(test_y,predict_y,average=None))

[0.07048458 0.78443114 0.53120464]


In [91]:
def predict(s):
    a=vector_text.transform(s)
    return (clf.predict(a))
a=predict(sent_test.modified_sent)
df= pd.DataFrame(data={"predictions":a})
df.to_csv('sentiments.csv')